In [9]:
from sedona.spark import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
import time

#Αρχικοποίηση SparkSession
spark = SparkSession.builder \
    .appName("Query 5 - Police Station Crimes/Distance") \
    .config("spark.executor.instances", "2") \
    .config("spark.executor.cores", "4") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

#Χρόνος εκτέλεσης για 2 executors με 4 cores/8 GB memory
start_time=time.time()

#Δημιουργία sedona context για χρήση geospatial δεδομένων
sedona = SedonaContext.create(spark)

#Φόρτωση δεδομένων εγκλημάτων από αρχεία CSV
file_path = 's3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv' 
file_path2 = 's3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv'
data1 = spark.read.csv(file_path, header=True, inferSchema=True)
data2 = spark.read.csv(file_path2, header=True, inferSchema=True)

#Ένωση των δύο συνόλων δεδομένων
crime_data = data1.union(data2)

#Δημιουργία γεωμετρικής στήλης από τα πεδία (LAT, LON)
crime_data = crime_data.withColumn("geom", expr("ST_Point(LON, LAT)"))

# Κανονικοποίηση της στήλης AREA (αφαίρεση μηδενικών μπροστά)
crime_data = crime_data.withColumn("AREA", col("AREA ").cast("int"))

#Φόρτωση δεδομένων αστυνομικών τμημ΄άτων από αρχείο CSV
file_path3 = 's3://initial-notebook-data-bucket-dblab-905418150721/LA_Police_Stations.csv' 
police_data = spark.read.csv(file_path3, header=True, inferSchema=True)

#Δημιουργία γεωμετρικής στήλης από τα πεδία (LAT, LON)
police_data = police_data.withColumn("PD_coords", expr("ST_Point(X, Y)"))

#Join βασισμένο στην ισότητα AREA (crime_data) και FID (police_data)
joined_df = crime_data.join(police_data, crime_data["AREA"] == police_data["FID"])

#Υπολογισμός αποστάσεων μεταξύ εγκλημάτων και αστυνομικών τμημάτων
joined_df = joined_df.withColumn("distance_km", expr("ST_DistanceSphere(geom, PD_coords) / 1000"))

#Ομαδοποίηση ανά τμήμα για υπολογισμό αριθμού εγκλημάτων και μέσης απόστασης
result = joined_df.groupBy("DIVISION").agg(count("DR_NO").alias("#"),avg("distance_km").alias("average_distance"))

#Ταξινόμηση κατά αριθμό εγκλημάτων σε φθίνουσα σειρά
final_result = result.select(
    col("DIVISION").alias("division"),
    col("average_distance"),
    col("#")
).orderBy(col("#").desc())

#Μέτρηση του χρόνου εκτέλεσης
end_time=time.time()

#Εμφάνιση αποτελεσμάτων
print(f"Execution time with 2 executors with 4 cores and 8GB memory: {end_time - start_time} seconds")
final_result.show(final_result.count(), truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time with 2 executors with 4 cores and 8GB memory: 3.408041477203369 seconds
+----------------+------------------+------+
|division        |average_distance  |#     |
+----------------+------------------+------+
|RAMPART         |21.35443425592832 |206981|
|77TH STREET     |16.146711450329732|192367|
|NORTHEAST       |41.567110485771856|171166|
|HARBOR          |51.07732689136336 |166946|
|NORTH HOLLYWOOD |16.209757573536077|164710|
|TOPANGA         |59.87173190508427 |161256|
|NEWTON          |44.32761502052693 |151053|
|HOLLYWOOD       |22.698919487730954|148886|
|DEVONSHIRE      |45.52117077681358 |145135|
|FOOTHILL        |20.90618014971378 |143777|
|OLYMPIC         |17.153219272101406|142833|
|HOLLENBECK      |37.91632560507371 |142327|
|MISSION         |21.236521554243375|138708|
|WEST VALLEY     |22.17753423796008 |138044|
|CENTRAL         |26.07495712884763 |136374|
|SOUTHEAST       |28.367697058440474|136104|
|WEST LOS ANGELES|13.881883428494104|134369|
|SOUTHWEST   

In [8]:
from sedona.spark import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
import time

#Αρχικοποίηση SparkSession
spark = SparkSession.builder \
    .appName("Query 5 - Police Station Crimes/Distance") \
    .config("spark.executor.instances", "4") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

#Χρόνος εκτέλεσης για 4 executors με 2 cores/4 GB memory
start_time=time.time()

#Δημιουργία sedona context για χρήση geospatial δεδομένων
sedona = SedonaContext.create(spark)

#Φόρτωση δεδομένων εγκλημάτων από αρχεία CSV
file_path = 's3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv' 
file_path2 = 's3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv'
data1 = spark.read.csv(file_path, header=True, inferSchema=True)
data2 = spark.read.csv(file_path2, header=True, inferSchema=True)

#Ένωση των δύο συνόλων δεδομένων
crime_data = data1.union(data2)

#Δημιουργία γεωμετρικής στήλης από τα πεδία (LAT, LON)
crime_data = crime_data.withColumn("geom", expr("ST_Point(LON, LAT)"))

# Κανονικοποίηση της στήλης AREA (αφαίρεση μηδενικών μπροστά)
crime_data = crime_data.withColumn("AREA", col("AREA ").cast("int"))

#Φόρτωση δεδομένων αστυνομικών τμημ΄άτων από αρχείο CSV
file_path3 = 's3://initial-notebook-data-bucket-dblab-905418150721/LA_Police_Stations.csv' 
police_data = spark.read.csv(file_path3, header=True, inferSchema=True)

#Δημιουργία γεωμετρικής στήλης από τα πεδία (LAT, LON)
police_data = police_data.withColumn("PD_coords", expr("ST_Point(X, Y)"))

#Join βασισμένο στην ισότητα AREA (crime_data) και FID (police_data)
joined_df = crime_data.join(police_data, crime_data["AREA"] == police_data["FID"])

#Υπολογισμός αποστάσεων μεταξύ εγκλημάτων και αστυνομικών τμημάτων
joined_df = joined_df.withColumn("distance_km", expr("ST_DistanceSphere(geom, PD_coords) / 1000"))

#Ομαδοποίηση ανά τμήμα για υπολογισμό αριθμού εγκλημάτων και μέσης απόστασης
result = joined_df.groupBy("DIVISION").agg(count("DR_NO").alias("#"),avg("distance_km").alias("average_distance"))

#Ταξινόμηση κατά αριθμό εγκλημάτων σε φθίνουσα σειρά
final_result = result.select(
    col("DIVISION").alias("division"),
    col("average_distance"),
    col("#")
).orderBy(col("#").desc())

#Μέτρηση του χρόνου εκτέλεσης
end_time=time.time()

#Εμφάνιση αποτελεσμάτων
print(f"Execution time with 4 executors with 2 cores and 4GB memory: {end_time - start_time} seconds")
final_result.show(final_result.count(), truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time with 4 executors with 2 cores and 4GB memory: 4.363903522491455 seconds
+----------------+------------------+------+
|division        |average_distance  |#     |
+----------------+------------------+------+
|RAMPART         |21.35443425592832 |206981|
|77TH STREET     |16.146711450329736|192367|
|NORTHEAST       |41.567110485771856|171166|
|HARBOR          |51.07732689136337 |166946|
|NORTH HOLLYWOOD |16.20975757353608 |164710|
|TOPANGA         |59.871731905084275|161256|
|NEWTON          |44.327615020526935|151053|
|HOLLYWOOD       |22.698919487730947|148886|
|DEVONSHIRE      |45.52117077681356 |145135|
|FOOTHILL        |20.90618014971378 |143777|
|OLYMPIC         |17.153219272101403|142833|
|HOLLENBECK      |37.91632560507371 |142327|
|MISSION         |21.236521554243375|138708|
|WEST VALLEY     |22.17753423796008 |138044|
|CENTRAL         |26.07495712884763 |136374|
|SOUTHEAST       |28.367697058440474|136104|
|WEST LOS ANGELES|13.8818834284941  |134369|
|SOUTHWEST   

In [7]:
from sedona.spark import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
import time

#Αρχικοποίηση SparkSession
spark = SparkSession.builder \
    .appName("Query 5 - Police Station Crimes/Distance") \
    .config("spark.executor.instances", "8") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

#Χρόνος εκτέλεσης για 8 executors με 1 core/2 GB memory
start_time=time.time()

#Δημιουργία sedona context για χρήση geospatial δεδομένων
sedona = SedonaContext.create(spark)

#Φόρτωση δεδομένων εγκλημάτων από αρχεία CSV
file_path = 's3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv' 
file_path2 = 's3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv'
data1 = spark.read.csv(file_path, header=True, inferSchema=True)
data2 = spark.read.csv(file_path2, header=True, inferSchema=True)

#Ένωση των δύο συνόλων δεδομένων
crime_data = data1.union(data2)

#Δημιουργία γεωμετρικής στήλης από τα πεδία (LAT, LON)
crime_data = crime_data.withColumn("geom", expr("ST_Point(LON, LAT)"))

# Κανονικοποίηση της στήλης AREA (αφαίρεση μηδενικών μπροστά)
crime_data = crime_data.withColumn("AREA", col("AREA ").cast("int"))

#Φόρτωση δεδομένων αστυνομικών τμημ΄άτων από αρχείο CSV
file_path3 = 's3://initial-notebook-data-bucket-dblab-905418150721/LA_Police_Stations.csv' 
police_data = spark.read.csv(file_path3, header=True, inferSchema=True)

#Δημιουργία γεωμετρικής στήλης από τα πεδία (LAT, LON)
police_data = police_data.withColumn("PD_coords", expr("ST_Point(X, Y)"))

#Join βασισμένο στην ισότητα AREA (crime_data) και FID (police_data)
joined_df = crime_data.join(police_data, crime_data["AREA"] == police_data["FID"])

#Υπολογισμός αποστάσεων μεταξύ εγκλημάτων και αστυνομικών τμημάτων
joined_df = joined_df.withColumn("distance_km", expr("ST_DistanceSphere(geom, PD_coords) / 1000"))

#Ομαδοποίηση ανά τμήμα για υπολογισμό αριθμού εγκλημάτων και μέσης απόστασης
result = joined_df.groupBy("DIVISION").agg(count("DR_NO").alias("#"),avg("distance_km").alias("average_distance"))

#Ταξινόμηση κατά αριθμό εγκλημάτων σε φθίνουσα σειρά
final_result = result.select(
    col("DIVISION").alias("division"),
    col("average_distance"),
    col("#")
).orderBy(col("#").desc())

#Μέτρηση του χρόνου εκτέλεσης
end_time=time.time()

#Εμφάνιση αποτελεσμάτων
print(f"Execution time with 8 executors with 1 core and 2GB memory: {end_time - start_time} seconds")
final_result.show(final_result.count(), truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time with 8 executors with 1 core and 2GB memory: 20.36761164665222 seconds
+----------------+------------------+------+
|division        |average_distance  |#     |
+----------------+------------------+------+
|RAMPART         |21.35443425592828 |206981|
|77TH STREET     |16.146711450329654|192367|
|NORTHEAST       |41.56711048577183 |171166|
|HARBOR          |51.0773268913635  |166946|
|NORTH HOLLYWOOD |16.20975757353609 |164710|
|TOPANGA         |59.871731905083806|161256|
|NEWTON          |44.32761502052695 |151053|
|HOLLYWOOD       |22.698919487731022|148886|
|DEVONSHIRE      |45.52117077681354 |145135|
|FOOTHILL        |20.90618014971369 |143777|
|OLYMPIC         |17.15321927210145 |142833|
|HOLLENBECK      |37.9163256050738  |142327|
|MISSION         |21.236521554243236|138708|
|WEST VALLEY     |22.17753423795999 |138044|
|CENTRAL         |26.074957128847693|136374|
|SOUTHEAST       |28.367697058440452|136104|
|WEST LOS ANGELES|13.881883428494   |134369|
|SOUTHWEST    